In [1]:
from pathlib import Path

import numpy as np

from model.factory import tts_ljspeech, tts_custom
from utils.config_manager import Config
from data.audio import Audio
from vocoding.predictors import HiFiGANPredictor, MelGANPredictor


    
    
  
        
    

2021-11-29 12:42:22.119862: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
vocoder = HiFiGANPredictor.from_folder('vocoding/hifigan/vctk')

In [ ]:
batch_size = 8
h = 33
w = 45
data = np.random.normal(1, 1, size=(batch_size, h, w))
data = torch.Tensor(data)
h0, c0 = torch.zeros(1, batch_size, w), torch.zeros(1, batch_size, w)

traced_script_module = torch.jit.trace(model, (data, h0,c0))
traced_script_module.save('model.pt')